In [1]:
from wdtools import *

C:\Users\clid1852\AppData\Anaconda3\envs\geoenv\lib\site-packages\geopandas\_compat.py:106: UserWarning: The Shapely GEOS version (3.11.1-CAPI-1.17.1) is incompatible with the GEOS version PyGEOS was compiled with (3.9.1-CAPI-1.14.2). Conversions between both will be slow.
  warnings.warn(


In [2]:
files = list_files(os.path.join(wdpath, 'Set001'))

In [3]:
files = [file for file in files if '~$' not in file]

In [4]:
set_ID = 'Set001'

In [5]:
start1 = time.time()
frames = []
for year in range(2017, 2023):
    #start2 = time.time()
    tx_dt = read_taxlot(year)
    #end2 = time.time()
    #print("it took {0} minutes to read taxlot in {1}".format(str(round((end2 - start2)/60, 0)), str(year)))
    for file in files:
        #print("processing {0} in {1}".format(file, str(year)))
        wd_dt = read_wd_table(setID = set_ID, file = file)  
        df = wd_dt[wd_dt.IDyear == str(year)].merge(tx_dt[['ORTaxlot', 'geometry']], 
                                                    on='ORTaxlot', 
                                                    how='left')
        frames.append(df)
        #print("processed {0} in {1}".format(file, str(year)))
df = pd.concat(frames, ignore_index=True)      
end1 = time.time()
print("it took {0} minutes to complete {1}".format(str(round((end1 - start1)/60, 0)), set_ID))

it took 16.0 minutes to complete Set001


In [6]:
df.columns

Index(['wetdet_delin_number', 'trsqq', 'parcel_id', 'address_location_desc',
       'city', 'county', 'site_name', 'site_desc', 'latitude', 'longitude',
       'Coord-Source', 'DocumentName', 'DecisionLink', 'is_batch_file',
       'status_name', 'received_date', 'response_date',
       'reissuance_response_date', 'project_id', 'site_id', 'record_ID',
       'lots', 'lot', 'cnt_code', 'ORTaxlot', 'year', 'IDyear', 'geometry'],
      dtype='object')

In [7]:
df.reissuance_response_date.dtype

dtype('float64')

In [8]:
df.head(3)

,wetdet_delin_number,trsqq,parcel_id,address_location_desc,city,county,site_name,site_desc,latitude,longitude,...,project_id,site_id,record_ID,lots,lot,cnt_code,ORTaxlot,year,IDyear,geometry
0,WD2017-0003,06N10W03DA,500 (partial),McCormick Rd .025 mi S of Hillila Rd,Gearhart,Clatsop,McCormick Rd 06N10W03DA,Residential,46.030197,-123.905217,...,NaN,103777,1,[500],500,4,0406.00N10.00W03DA--000000500,2017,2017,"MULTIPOLYGON (((448821.559 1578307.121, 448814..."
1,WD2017-0033,08N08W25,101,"W of 92260 Koppisch Rd, Astoria",Astoria,Clatsop,08N08W25,site evaluation,46.155778,-123.610751,...,NaN,104259,2,[101],101,4,0408.00N08.00W2500--000000101,2017,2017,"MULTIPOLYGON (((524028.003 1621190.023, 524220..."
2,WD2017-0056,08N10W09B,"2500, ROWs (both partial)",Warrenton Dr and NW 17th St,Warrenton,Clatsop,Gann Dev 08N10W09B,residential development,46.190638,-123.932177,...,NaN,104411,3,[2500],2500,4,0408.00N10.00W09B0--000002500,2017,2017,"MULTIPOLYGON (((443198.754 1636916.627, 443123..."


In [9]:
df['response_date'] = df['response_date'].dt.strftime("%Y-%m-%d")

In [10]:
df['received_date'] = df['received_date'].dt.strftime("%Y-%m-%d")

In [13]:
df['lots'] = df['lots'].apply(lambda x: ' '.join(dict.fromkeys(x).keys()))

In [14]:
gdf = gpd.GeoDataFrame(df, crs="EPSG:2992", geometry='geometry')

In [17]:
gdf.rename(columns={'wetdet_delin_number': 'wdID', 
                      'address_location_desc':'loc_desc', 
                      'Coord-Source': 'coord_src',
                      'DocumentName':'doc_name',
                      'DecisionLink':'doc_link',
                      'is_batch_file':'isbatfile',
                      'status_name': 'status_nm',
                      'received_date':'receiveddt', 
                      'response_date':'responsedt',
                      'reissuance_response_date':'reissuance', 
                      }, inplace=True)

In [18]:
gdf.to_file(os.path.join(r'T:\DCProjects\EPA-WD', 'matched_records_with_lotID'))